In [5]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import re
from gensim.models import Word2Vec
import nltk
import numpy as np

###### Load Dataset

In [2]:
train = pd.read_csv("dataset/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv( "dataset/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

In [8]:
def split_sentence(sentence):
    reg1 = re.compile(r"\x3C.{1,8}\x3E")
    reg2 = re.compile("[^a-zA-Z0-9]")
    a = re.sub(reg1,"",sentence)
    b = re.sub(reg2," ",a)
    return b.lower().split()

In [9]:
def pre_process(para):
    tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
    sentences = list()
    paragraphs = tokenizer.tokenize(para)
    for sentence in paragraphs:
        if len(sentence) > 1:
            sentences.append(split_sentence(sentence))
    return sentences

###### Building sentence list for Word2Vec

In [10]:
sentences = list()

for review in train["review"].values:
    sentences += (pre_process(review))

for review in unlabeled_train["review"].values:
    sentences += (pre_process(review))

In [12]:
cpu_threads = 8
dimension = 50
min_count = 40
window = 10

###### Building Word2Vec Model

In [13]:
model = Word2Vec(sentences, workers=cpu_threads, size=dimension, min_count = min_count, window = window, sample = 1e-3)
model.init_sims(replace=True)

model_name = "imdb_word2Vec_50"
model.save(model_name)